In [1]:
import matplotlib.pyplot as plt 
import networkx as nx
import pandas as pd 

from pgmpy.utils import get_example_model
from pgmpy.inference import BeliefPropagation

from IPython.display import display, Image
%config IPCompleter.greedy=True


c:\Users\Hasan\Desktop\PythonProj\genius-apps\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Problem Description:** You are tasked with building a Bayesian network model to assist in the diagnosis of respiratory diseases in the Asian population. The network should represent causal relationships between diseases, symptoms, and risk factors, allowing physicians to infer the likelihood of a patient having a particular disease based on observed symptoms and risk factors.

In [7]:
asia_model = get_example_model('asia')

bif_file_path = 'asia.bif'

with open(bif_file_path, 'r') as file:
    for line in file:
        print(line.strip())

network unknown {
}
variable asia {
type discrete [ 2 ] { yes, no };
}
variable tub {
type discrete [ 2 ] { yes, no };
}
variable smoke {
type discrete [ 2 ] { yes, no };
}
variable lung {
type discrete [ 2 ] { yes, no };
}
variable bronc {
type discrete [ 2 ] { yes, no };
}
variable either {
type discrete [ 2 ] { yes, no };
}
variable xray {
type discrete [ 2 ] { yes, no };
}
variable dysp {
type discrete [ 2 ] { yes, no };
}
probability ( asia ) {
table 0.01, 0.99;
}
probability ( tub | asia ) {
(yes) 0.05, 0.95;
(no) 0.01, 0.99;
}
probability ( smoke ) {
table 0.5, 0.5;
}
probability ( lung | smoke ) {
(yes) 0.1, 0.9;
(no) 0.01, 0.99;
}
probability ( bronc | smoke ) {
(yes) 0.6, 0.4;
(no) 0.3, 0.7;
}
probability ( either | lung, tub ) {
(yes, yes) 1.0, 0.0;
(no, yes) 1.0, 0.0;
(yes, no) 1.0, 0.0;
(no, no) 0.0, 1.0;
}
probability ( xray | either ) {
(yes) 0.98, 0.02;
(no) 0.05, 0.95;
}
probability ( dysp | bronc, either ) {
(yes, yes) 0.9, 0.1;
(no, yes) 0.7, 0.3;
(yes, no) 0.8, 0.2;

In [ ]:
edges = asia_model.edges()
nodes = asia_model.nodes()

G = nx.DiGraph()
G.add_edges_from(edges)

# plt.figure(figsize=(8, 6))
# pos = nx.spring_layout(G)
# nx.draw(G, pos, with_labels=True, node_size=2000, node_color="skyblue", font_size=12, font_weight="bold", arrowsize=20)
# plt.title("Asia Model")
# plt.show()

#display(Image(filename='./images/asia.png'))

In [12]:
bp = BeliefPropagation(asia_model)
query_result = bp.query(variables=["asia", "smoke"], evidence={"lung": "yes"})
print(query_result)

+---------+------------+-------------------+
| asia    | smoke      |   phi(asia,smoke) |
+=========+============+===================+
| asia(0) | smoke(yes) |            0.0091 |
+---------+------------+-------------------+
| asia(0) | smoke(no)  |            0.0009 |
+---------+------------+-------------------+
| asia(1) | smoke(yes) |            0.9000 |
+---------+------------+-------------------+
| asia(1) | smoke(no)  |            0.0900 |
+---------+------------+-------------------+


c:\Users\Hasan\Desktop\PythonProj\genius-apps\env\Lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:796: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi1.values


# **Learning** 

In [8]:
import numpy as np
from pgmpy.estimators import MaximumLikelihoodEstimator

In [9]:
asia_model = get_example_model('asia')

edges = asia_model.edges()
nodes = asia_model.nodes()

values = pd.DataFrame(np.random.randint(low=0, high=2, size=(1000, len(nodes))),
                      columns=nodes)

estimator = MaximumLikelihoodEstimator(asia_model, values)

print(estimator.get_parameters())

for cpd in estimator.get_parameters():
    print(cpd)

[<TabularCPD representing P(asia:2) at 0x1e1430e5450>, <TabularCPD representing P(tub:2 | asia:2) at 0x1e11f62fb10>, <TabularCPD representing P(smoke:2) at 0x1e11e83f610>, <TabularCPD representing P(lung:2 | smoke:2) at 0x1e11e8465d0>, <TabularCPD representing P(bronc:2 | smoke:2) at 0x1e11e847c50>, <TabularCPD representing P(either:2 | lung:2, tub:2) at 0x1e11e847b50>, <TabularCPD representing P(xray:2 | either:2) at 0x1e11e8472d0>, <TabularCPD representing P(dysp:2 | bronc:2, either:2) at 0x1e11e846110>]
+---------+-------+
| asia(0) | 0.488 |
+---------+-------+
| asia(1) | 0.512 |
+---------+-------+
+--------+---------------------+------------+
| asia   | asia(0)             | asia(1)    |
+--------+---------------------+------------+
| tub(0) | 0.47950819672131145 | 0.53515625 |
+--------+---------------------+------------+
| tub(1) | 0.5204918032786885  | 0.46484375 |
+--------+---------------------+------------+
+----------+-------+
| smoke(0) | 0.477 |
+----------+-------+
| s